# Getting AWS ECR credentials into Kubernetes

This is an imaginary response from the following command:

```sh
aws ecr get-authorization-token --registry-ids "111111111111" --region "us-east-1"
aws ecr get-authorization-token --registry-ids "222222222222" --region "us-east-1"
aws ecr get-authorization-token --registry-ids "222222222222" --region "us-west-2"
```

It's output looks something like this (account id-s and token data got sanitized for this exercise):

In [1]:
<input.json jq

{
  "authorizationData": [
    {
      "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
      "expiresAt": 158164069.791,
      "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
    }
  ]
}
{
  "authorizationData": [
    {
      "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
      "expiresAt": 158164069.791,
      "proxyEndpoint": "https://222222222222.dkr.ecr.us-east-1.amazonaws.com"
    }
  ]
}
{
  "authorizationData": [
    {
      "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRkOQ==",
     

We want to produce from this a single kubernetes secret that would look something like this:

In [2]:
<desired.json jq

{
  "apiVersion": "v1",
  "kind": "Secret",
  "type": "kubernetes.io/dockerconfigjson",
  "metadata": {
    "name": "aws-creds"
  },
  "data": {
    ".dockerconfigjson": "eyJhdXRocyI6eyJodHRwczovLzExMTExMTExMTExMS5ka3IuZWNyLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tIjp7InVzZXJuYW1lIjoiQVdTIiwicGFzc3dvcmQiOiJleUp3WVhsc2IyRmtJam9pV0ZoWVdGaFlXRmdpTENKa1lYUmhhMlY1SWpvaVdWbFpXVmxaV1ZraUxDSjJaWEp6YVc5dUlqb2lNaUlzSW5SNWNHVWlPaUpFUVZSQlgwdEZXU0lzSW1WNGNHbHlZWFJwYjI0aU9qRTFPREF5TnpNd05EWjkiLCJlbWFpbCI6Im5vbmUiLCJhdXRoIjoiUVZkVE9tVjVTbmRaV0d4ellqSkdhMGxxYjJsWFJtaFpWMFpvV1ZkR1oybE1RMHByV1ZoU2FHRXlWalZKYW05cFYxWnNXbGRXYkZwWFZtdHBURU5LTWxwWVNucGhWemwxU1dwdmFVMXBTWE5KYmxJMVkwZFZhVTlwU2tWUlZsSkNXREIwUmxkVFNYTkpiVlkwWTBkc2VWbFlVbkJpTWpScFQycEZNVTlFUVhsT2VrMTNUa1JhT1E9PSJ9LCJodHRwczovLzIyMjIyMjIyMjIyMi5ka3IuZWNyLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tIjp7InVzZXJuYW1lIjoiQVdTIiwicGFzc3dvcmQiOiJleUp3WVhsc2IyRmtJam9pV0ZoWVdGaFlXRmdpTENKa1lYUmhhMlY1SWpvaVdWbFpXVmxaV1ZraUxDSjJaWEp6YVc5dUlqb2lNaUlzSW5SNWNHVWlPaUpFUVZSQlgwdEZXU

## Inspecting the input

First we create a directory for our temporary data:

In [3]:
mkdir -p out

Let's start inspecting the first token we get:

In [4]:
<input.json jq -s '[.[][][]][0]' | tee out/1.json | jq

{
  "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
  "expiresAt": 158164069.791,
  "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
}


 This token seems base64 encoded, let's see what it contains:

In [5]:
<out/1.json jq '.authorizationToken |= @base64d' | tee out/2.json | jq

{
  "authorizationToken": "AWS:eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
  "expiresAt": 158164069.791,
  "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
}


It seems like it's some AWS token!
If we inspect the output of the `aws ecr get-login` command, we'll see that the username is "AWS", so that second part must be the password we can use for docker login.
Let's see if the second part is base64 encoding:

In [6]:
<out/2.json jq '.authorizationToken |= (split(":")[1]|@base64d)' | tee out/3.json | jq

{
  "authorizationToken": "{\"payload\":\"XXXXXXXX\",\"datakey\":\"YYYYYYYY\",\"version\":\"2\",\"type\":\"DATA_KEY\",\"expiration\":1580273046}",
  "expiresAt": 158164069.791,
  "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
}


Seems like it's some sort of JSON payload. Let's parse it.

In [7]:
<out/3.json jq '.authorizationToken |= fromjson' | tee out/4.json | jq

{
  "authorizationToken": {
    "payload": "XXXXXXXX",
    "datakey": "YYYYYYYY",
    "version": "2",
    "type": "DATA_KEY",
    "expiration": 1580273046
  },
  "expiresAt": 158164069.791,
  "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
}


This json data seems to be the keys to the kingdom. We didn't extract anything useful, but we know it's some sort of token in JSON format.

## Finding out about the kubernetes imagepullsecret format

If we inspect the format of `aws ecr get-login`, we'll see something like this:
```sh
docker login -u AWS -p eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9 -e none https://111111111111.dkr.ecr.us-east-1.amazonaws.com
```
That shows that:
- username is `AWS`
- password is a token
- email is `none`
-registry url is `https://111111111111.dkr.ecr.us-east-1.amazonaws.com`

We can learn how to structure imagePullSecrets for Kubernetes from the official docs:
https://kubernetes.io/docs/tasks/configure-pod-container/pull-image-private-registry/

a credential can be created with the following kubectl command:

In [8]:
kubectl create secret docker-registry  aws-creds \
    --docker-server=https://example.com \
    --docker-username=AWS \
    --docker-password=xxxyyyzzz \
    --docker-email=none -o json --dry-run | tee out/5.json | jq

{
  "kind": "Secret",
  "apiVersion": "v1",
  "metadata": {
    "name": "aws-creds",
    "creationTimestamp": null
  },
  "data": {
    ".dockerconfigjson": "eyJhdXRocyI6eyJodHRwczovL2V4YW1wbGUuY29tIjp7InVzZXJuYW1lIjoiQVdTIiwicGFzc3dvcmQiOiJ4eHh5eXl6enoiLCJlbWFpbCI6Im5vbmUiLCJhdXRoIjoiUVZkVE9uaDRlSGw1ZVhwNmVnPT0ifX19"
  },
  "type": "kubernetes.io/dockerconfigjson"
}


Let's inspect how that .dockerconfigjson looks like:

In [9]:
<out/5.json jq '.data |= with_entries({key, value: .value|@base64d|fromjson})'

{
  "kind": "Secret",
  "apiVersion": "v1",
  "metadata": {
    "name": "aws-creds",
    "creationTimestamp": null
  },
  "data": {
    ".dockerconfigjson": {
      "auths": {
        "https://example.com": {
          "username": "AWS",
          "password": "xxxyyyzzz",
          "email": "none",
          "auth": "QVdTOnh4eHl5eXp6eg=="
        }
      }
    }
  },
  "type": "kubernetes.io/dockerconfigjson"
}


We will have to produce exactly that format from the `aws ecr get-authorization-token` output.
The `auth` field if we inspect it is actually the "AWS:xxxyyyzzz" pair.


## Transforming ECR Auth Token into Kubernetes dockerconfig json format

Let's again look at our tokens we get from aws ecr:

In [10]:
<input.json jq

{
  "authorizationData": [
    {
      "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
      "expiresAt": 158164069.791,
      "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
    }
  ]
}
{
  "authorizationData": [
    {
      "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
      "expiresAt": 158164069.791,
      "proxyEndpoint": "https://222222222222.dkr.ecr.us-east-1.amazonaws.com"
    }
  ]
}
{
  "authorizationData": [
    {
      "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRkOQ==",
     

That shows two things:
1. We need to merge the two objects into a single one
2. We need to extract the username and the password for each record and add it
3. We need to set the email to 'none'.

Let's do that!
First We'll shave off the boilerplate:

In [11]:
<input.json jq -s 'map(.[][])' | tee out/6.json | jq

[
  {
    "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
    "expiresAt": 158164069.791,
    "proxyEndpoint": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com"
  },
  {
    "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ==",
    "expiresAt": 158164069.791,
    "proxyEndpoint": "https://222222222222.dkr.ecr.us-east-1.amazonaws.com"
  },
  {
    "authorizationToken": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRkOQ==",
    "expiresAt": 158164069.791,
    "proxyEndpoint": "https://222222222222.dkr.ecr.us-west-2.amazonaws.com"
  }
]


Next we try to massage the data towards the desired dockerconfig json format.
First we transform the object into an [entry](https://stedolan.github.io/jq/manual/#to_entries,from_entries,with_entries).


In [12]:
<out/6.json jq 'map({key: .proxyEndpoint, value: { auth: .authorizationToken}})' | tee out/7.json | jq

[
  {
    "key": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com",
    "value": {
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
    }
  },
  {
    "key": "https://222222222222.dkr.ecr.us-east-1.amazonaws.com",
    "value": {
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
    }
  },
  {
    "key": "https://222222222222.dkr.ecr.us-west-2.amazonaws.com",
    "value": {
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRkOQ=="
    }
  }
]


Next, we split out the username and password parts from the auth token.
First we take the token, base64 decode it, to get it into the `username:password` format.
Next, we construct a new object for `value`, that contains `username`, `password` and `email` fields
besides the `auth` field.

In [13]:
<out/7.json jq '
map(
    .value |= (.auth|@base64d|split(":")) as $auth
            | {
                username: $auth[0],
                password: $auth[1],
                email: "none",
                auth
            }
)' | tee out/8.json | jq

[
  {
    "key": "https://111111111111.dkr.ecr.us-east-1.amazonaws.com",
    "value": {
      "username": "AWS",
      "password": "eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
      "email": "none",
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
    }
  },
  {
    "key": "https://222222222222.dkr.ecr.us-east-1.amazonaws.com",
    "value": {
      "username": "AWS",
      "password": "eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
      "email": "none",
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
    }
  }

Now comes the magic: we use `from_entries` to convert the list of objects with 'key' and 'value' pairs into a single object.

In [14]:
<out/8.json jq 'from_entries' | tee out/9.json | jq

{
  "https://111111111111.dkr.ecr.us-east-1.amazonaws.com": {
    "username": "AWS",
    "password": "eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
    "email": "none",
    "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
  },
  "https://222222222222.dkr.ecr.us-east-1.amazonaws.com": {
    "username": "AWS",
    "password": "eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
    "email": "none",
    "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
  },
  "https://222222222222.dkr.ecr.us-west-2.amazonaws.com": {
    "username": "A

Finally, we wrap the entire object into `auths`.

In [15]:
<out/9.json jq '{auths: .}' | tee out/10.json | jq

{
  "auths": {
    "https://111111111111.dkr.ecr.us-east-1.amazonaws.com": {
      "username": "AWS",
      "password": "eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
      "email": "none",
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
    },
    "https://222222222222.dkr.ecr.us-east-1.amazonaws.com": {
      "username": "AWS",
      "password": "eyJwYXlsb2FkIjoiWFhYWFhYWFgiLCJkYXRha2V5IjoiWVlZWVlZWVkiLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1ODAyNzMwNDZ9",
      "email": "none",
      "auth": "QVdTOmV5SndZWGxzYjJGa0lqb2lXRmhZV0ZoWVdGZ2lMQ0prWVhSaGEyVjVJam9pV1ZsWldWbFpXVmtpTENKMlpYSnphVzl1SWpvaU1pSXNJblI1Y0dVaU9pSkVRVlJCWDB0RldTSXNJbVY0Y0dseVlYUnBiMjRpT2pFMU9EQXlOek13TkRaOQ=="
    },
    "https://222222222222.dkr.ecr.us-west

## Generating the kubernetes Secret manifest

In [16]:
<out/10.json jq '{
apiVersion: "v1",
kind: "Secret",
type: "kubernetes.io/dockerconfigjson",
metadata: {
    name: "aws-creds"
},
data: {
    ".dockerconfigjson": .|tojson|@base64
}
}' | tee out/11.json | jq

{
  "apiVersion": "v1",
  "kind": "Secret",
  "type": "kubernetes.io/dockerconfigjson",
  "metadata": {
    "name": "aws-creds"
  },
  "data": {
    ".dockerconfigjson": "eyJhdXRocyI6eyJodHRwczovLzExMTExMTExMTExMS5ka3IuZWNyLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tIjp7InVzZXJuYW1lIjoiQVdTIiwicGFzc3dvcmQiOiJleUp3WVhsc2IyRmtJam9pV0ZoWVdGaFlXRmdpTENKa1lYUmhhMlY1SWpvaVdWbFpXVmxaV1ZraUxDSjJaWEp6YVc5dUlqb2lNaUlzSW5SNWNHVWlPaUpFUVZSQlgwdEZXU0lzSW1WNGNHbHlZWFJwYjI0aU9qRTFPREF5TnpNd05EWjkiLCJlbWFpbCI6Im5vbmUiLCJhdXRoIjoiUVZkVE9tVjVTbmRaV0d4ellqSkdhMGxxYjJsWFJtaFpWMFpvV1ZkR1oybE1RMHByV1ZoU2FHRXlWalZKYW05cFYxWnNXbGRXYkZwWFZtdHBURU5LTWxwWVNucGhWemwxU1dwdmFVMXBTWE5KYmxJMVkwZFZhVTlwU2tWUlZsSkNXREIwUmxkVFNYTkpiVlkwWTBkc2VWbFlVbkJpTWpScFQycEZNVTlFUVhsT2VrMTNUa1JhT1E9PSJ9LCJodHRwczovLzIyMjIyMjIyMjIyMi5ka3IuZWNyLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tIjp7InVzZXJuYW1lIjoiQVdTIiwicGFzc3dvcmQiOiJleUp3WVhsc2IyRmtJam9pV0ZoWVdGaFlXRmdpTENKa1lYUmhhMlY1SWpvaVdWbFpXVmxaV1ZraUxDSjJaWEp6YVc5dUlqb2lNaUlzSW5SNWNHVWlPaUpFUVZSQlgwdEZXU